In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

In [3]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [4]:
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train['feature_stock_id_sum'] = train['feature_41'] + train['feature_42'] + train['feature_43']
train['feature_1_2_cross'] = train['feature_1']/(train['feature_2']+1e-5)
train.fillna(-9999,inplace=True)


features = [c for c in train.columns if "feature" in c]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [8]:
import os, sys, gc, time, warnings, pickle, psutil, random
params={"num_leaves":xx,
        "feature_fraction":xx,
        "bagging_fraction":xx,
        "objective":"binary",
        "learning_rate":xx,
        "boosting_type":"gbdt",
        "metric":"auc",
        'device':'gpu',
        'seed':xx
    
       }
models = []
for i in range(5):
    model_path = '../input/folder_name/lgb_model_'+str(i)+'.bin' 
    clf = pickle.load(open(model_path, 'rb'))
    models.append(clf)

In [10]:
f = np.median
th = 0.5000
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        test_df['feature_stock_id_sum'] = test_df['feature_41'] + test_df['feature_42'] + test_df['feature_43']
        test_df['feature_1_2_cross'] = test_df['feature_1']/(test_df['feature_2']+1e-5)
        test_df.fillna(-9999,inplace=True)
        x_tt = test_df.loc[:, features].values
        pred = np.stack([model.predict(x_tt) for model in models]).T
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

15219it [03:23, 74.80it/s]
